In [23]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os

In [24]:
mainModel="../../Model/Weights/lr_dec_BiDir6200.pth"

In [25]:
import json

# Open and read the JSON file
with open('hyperparameters.json', 'r') as file:
    data = json.load(file)

# Print the data
print(data)


{'../../Model/Weights/lr_dec_BiDir6200.pth': {'lookback_window': 10, 'input_size': 6, 'output_size': 1, 'num_layers': 1, 'embed_dim': 64, 'hidden_size': 128, 'bidirectional': True}}


In [26]:
lookback_window = data[mainModel]['lookback_window']
input_size = data[mainModel]['input_size']
output_size = data[mainModel]['output_size']
num_layers = data[mainModel]['num_layers']
embed_dim = data[mainModel]['embed_dim']
hidden_size = data[mainModel]['hidden_size']
bidirectional = data[mainModel]['bidirectional']

In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using {device}")

Using cuda


In [28]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X_shuffled, y_shuffled, name_shuffled, open_shuffled):
        self.dataX = torch.tensor(X_shuffled, dtype=torch.float32)
        self.dataY = torch.tensor(y_shuffled, dtype=torch.float32)
        self.names = name_shuffled
        self.open = torch.tensor(open_shuffled, dtype=torch.float32)
        
    def __len__(self):
        return self.dataX.size(0)

    def __getitem__(self, idx):
        sequence = self.dataX[idx, :]
        target = self.dataY[idx]
        name = self.names[idx]
        open = self.open[idx]
        return sequence, target, name, open
    
import torch.nn.functional as F

class AttentionMechanism(nn.Module):
    def __init__(self, hidden_dim):
        super(AttentionMechanism, self).__init__()
        self.attention_weights = nn.Linear(hidden_dim, 1, bias=False)
    
    def forward(self, lstm_outputs):
        # lstm_outputs: [batch_size, seq_len, hidden_dim]
        attention_scores = self.attention_weights(lstm_outputs).squeeze(-1)  # [batch_size, seq_len]
        attention_weights = F.softmax(attention_scores, dim=1)  # [batch_size, seq_len]
        weighted_output = torch.sum(lstm_outputs * attention_weights.unsqueeze(-1), dim=1)  # [batch_size, hidden_dim]
        return weighted_output, attention_weights
    
class LSTMDoubleAttentionModel(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, output_dim, num_layers=1, bidirectional=False):
        super(LSTMDoubleAttentionModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.directions = 2 if bidirectional else 1

        # Embedding layer (optional: use if input is categorical or needs projection)
        self.embedding = nn.Linear(input_dim, embed_dim)  # Replace with nn.Embedding if needed
        
        # Encoder LSTM
        self.encoder_lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, 
                                    batch_first=True, bidirectional=bidirectional)
        
        # Attention
        self.attention = AttentionMechanism(hidden_dim * self.directions)
        
        # Decoder LSTM
        self.decoder_lstm = nn.LSTM(hidden_dim * self.directions, hidden_dim, num_layers,
                                    batch_first=True, bidirectional=bidirectional)
        
        # Fully Connected Layer
        self.fc = nn.Linear(hidden_dim * self.directions, output_dim)
    
    def forward(self, x):
        # x shape: [batch_size, seq_len, input_dim]
        
        # Embedding
        embedded = self.embedding(x)  # [batch_size, seq_len, embed_dim]
        
        # Encoder LSTM
        encoder_out, _ = self.encoder_lstm(embedded)  # [batch_size, seq_len, hidden_dim * directions]
        
        # Attention
        attention_out, attention_weights = self.attention(encoder_out)  # [batch_size, hidden_dim * directions]
        
        # Prepare Decoder Input (sequence length 1)
        decoder_input = attention_out.unsqueeze(1)  # [batch_size, 1, hidden_dim * directions]
        
        # Decoder LSTM
        decoder_out, _ = self.decoder_lstm(decoder_input)  # [batch_size, 1, hidden_dim * directions]
        
        # Final Prediction
        output = self.fc(decoder_out.squeeze(1))  # [batch_size, output_dim]
        
        return output, attention_weights

In [29]:
model = LSTMDoubleAttentionModel(input_size, embed_dim, hidden_size, output_size, num_layers=num_layers, bidirectional=bidirectional)

In [31]:
import sys

# Determine the parent directory of traderPackage by going two levels up
# from the current working directory (which is traderPackage/API)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

    
from traderPackage import Scaler
from sklearn.preprocessing import StandardScaler

model.load_state_dict(torch.load(mainModel, weights_only=True))
model.eval()


saveName=mainModel.split("/")[-1].split(".")[0]
saveFolder=f"../../Model/Weights/scalers"

import pickle
dbfile = open(os.path.join(saveFolder,f"{saveName}/scaler_{saveName}.pkl"), 'rb')    
scaler = pickle.load(dbfile)
dbfile.close()

dbfile = open(os.path.join(saveFolder,f"{saveName}/zScalerDic_{saveName}.pkl"), 'rb')    
zScalerDic= pickle.load(dbfile)
dbfile.close()